In [2]:
from jupyter_plotly_dash import JupyterDash
/home/gabriel/.config
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import os


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIXED ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter
import base64


###########################
# Data Manipulation / Model
###########################
# FIXED ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Adding Image asset for logo
image_file = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_file, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Animal Shelter Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'25%','width':'25%'})),
    html.Center(html.H2('CS340 Gabriel Romero')),
    html.Hr(),
    html.Br(),
    html.H3('Filter Options: '),
    html.Div(className='row',
            style={'display':'flex'},
                children=[
                    # Radio Buttons for filter options
                    dcc.RadioItems(
                        id='radio-id',
                              options = [
                                  {'label':'Water Rescue', 'value':'WR'},
                                  {'label':'Mountain or Wilderness Rescue', 'value':'MWLDR'},
                                  {'label':'Disaster Rescue or Individual Tracking', 'value':'DRIT'},
                                  {'label':'Reset Filter', 'value':'RF'},
                              ],
                        value= 'ALL', # Default value for validation check
                    )
                ]),
    html.Br(),
    html.Br(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXED: Set up the features for your interactive data table to make it user-friendly for your client
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
        style={'display':'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )]
    ),
    html.Center("Gabriel Romero SNHU CS340 Module 5")
])

#############################################
# Interaction Between Components / Controller
############################################
# Pie Chart callback
@app.callback(Output('graph-id','children'),
              [Input('datatable-id','derived_viewport_data')])
           
def generate_chart(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure = px.pie(df, names='breed', title='Percentage of breeds')
        )
    ]
    
# Datatable filter callback
@app.callback(Output('datatable-id', 'data'),
              [Input('radio-id', 'value')])
def update_dashboard(value): 
    # Validation check and filter check using value passed from Radio buttons
    if (value == 'ALL' or value == 'RF'):
        df = pd.DataFrame.from_records(shelter.read({}))
    # Water Rescue Query    
    elif (value == 'WR'):
        df = pd.DataFrame(list(shelter.read({'$and':[
            {'$or':[{'breed':'Labrador Retriever Mix'},{'breed':'Chesapeake Bay Retriever'},{'breed':'Newfoundland'}]},
            {'sex_upon_outcome':'Intact Female'},{'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}}
        ]})))      
    # Mountain and Wilderness rescue   
    elif (value == 'MWLDR'):
        df = pd.DataFrame(list(shelter.read({'$and':[
            {'$or':[{'breed':'German Shepherd'},{'breed':'Alaskan Malamute'},{'breed':'Old English Sheepdog'},{'breed':'Siberian Husky'},{'breed':'Rottweiler'}]},
            {'sex_upon_outcome':'Intact Male'},{'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}}
        ]})))
    # Disaster Rescue and Individual Tracking   
    elif (value == 'DRIT'):
        df = pd.DataFrame(list(shelter.read({'$and':[
            {'$or':[{'breed':'German Shepherd'},{'breed':'Doberman Pinscher'},{'breed':'Golden Retriever'},{'breed':'Bloodhound'},{'breed':'Rottweiler'}]},
            {'sex_upon_outcome':'Intact Male'},{'age_upon_outcome_in_weeks':{'$gte':20,'$lte':200}}
        ]})))
    
    data = df.to_dict('records')
    return data

#This callback will highlight a row on the data table when the user selects it   
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX at [30.75, -97.48]
    return [
        dl.Map(style={'width':'1000px','height':'500px'},
               center=[30.75,-97.48],
               zoom=10,
               children=[
                        dl.TileLayer(id="base-layer-id"),
                        dl.Marker(position=[30.75,-97.48], 
                                  children=[
                                            dl.Tooltip(dff.iloc[0,4]),
                                            dl.Popup([
                                                    html.H1("Animal Name"),
                                                    html.P(dff.iloc[1,9])
                                            ])
                                  ]           
                        )
              ]
       )
    ]

app